# 화장품 리뷰 분석

코드를 진행함에 있어 konlpy를 사용하기 위해 java jdk를 설치 해야 됨

pip install konlpy

pip install Cucco

pip install wordcloud  - visual studio 설치 되어 있어야 함

In [ ]:
import scipy as sp
import pandas as pd
import numpy as np
import nltk
import pickle

%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz             
import matplotlib.font_manager as fm        
import matplotlib as mpl
import warnings  

from konlpy.tag import Kkma       ; kkma = Kkma()
from konlpy.tag import Hannanum   ; hannanum = Hannanum()
from konlpy.tag import Okt        ; t = Okt()
from konlpy.tag import *

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt; t = Okt()
from sklearn.tree import export_graphviz

# 그래프 문자 깨지는 것 대처
plt.rc('font', family='NanumGothic')
mpl.rcParams['axes.unicode_minus'] = False

# 워닝 없애주는 것
warnings.filterwarnings('ignore')

In [ ]:
df_r= pd.read_excel('cos38.xlsx') # cos38은 화장품 홈페이지에서 리뷰를 크롤링한 파일
df_r.content

In [ ]:
print(df_r.shape)
df_r.head(5)  # df_r에서 5개의 행만을 확인

In [ ]:
df_r= pd.read_excel('cos38.xlsx')

In [ ]:
print(df_r.shape)
print(df_r.rating)

In [ ]:
print(df_r.content)

In [ ]:
content_list = df_r.content.values.tolist() # df_r의 리뷰들을 리스트 하나에 합쳐서 넣기

In [ ]:
content_list

In [ ]:
len(content_list)

In [ ]:
content_text = ''

for each_line in content_list:
    content_text = content_text + each_line + '\n'   # 리스트 하나로 합쳤던 문장들 사이에 공백을 개행문자로 변경

In [ ]:
content_text

In [ ]:
tokens_ko = t.morphs(content_text)
tokens_ko

In [ ]:
ko = nltk.Text(tokens_ko)   
print(len(ko.tokens))          # 토큰 전체 갯수
print(len(set(ko.tokens)))     # 토큰 unique 갯수

In [ ]:
ko.vocab().most_common(100)    # 가장 많이 나온 단어 100개

In [ ]:
# 불용어 : 인터넷 검색 시 검색 용어로 사용하지 않는 단어. 관사, 전치사, 조사, 접속사 등 검색 색인 단어로 의미가 없는 단어
stop_words = ['.','가',"!",'\r\n\r\n','\r\n','\n','\n ','요','답변','...','을','수','에','질문','제','를','이','도',
                      '좋','1','는','로','으로','2','것','은','다',',','니다','대','들',
                      '2017','들','데','..','의','때','겠','고','게','네요','한','일','할',
                      '10','?','하는','06','주','려고','인데','거','좀','는데','~','ㅎㅎ',
                      '하나','이상','20','뭐','까','있는','잘','습니다','다면','했','주려',
                      '지','있','못','후','중','줄','6','과','어떤','기본','!!',
                      '단어','라고','중요한','합','가요','....','보이','네','무지']

tokens_ko = [each_word for each_word in tokens_ko
             if each_word not in stop_words]

ko = nltk.Text(tokens_ko)
ko.vocab().most_common(50)   # 자주 사용된 50가지 단어 확인

In [ ]:
# PC에 설치되어 있는 폰트 리스트를 모두 불러오기

import matplotlib.font_manager as fm

font_list = fm.findSystemFonts(fontpaths = None, fontext = 'ttf')

font_list[:]

In [ ]:
# 그래프에서 한글 글자 및 마이너스 부호 깨짐 방지

import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False   
#그래프에서 마이너스 기호가 표시되도록 하는 설정입니다. 

In [ ]:
plt.figure(figsize=(15,6))
ko.plot(50) 
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
from PIL import Image

In [ ]:
data = ko.vocab().most_common(300)
data

In [ ]:
# 워드클라우드를 그려보자
wordcloud = WordCloud(font_path='c:/Windows/Fonts/malgun.ttf',
                      relative_scaling = 0.2,
                      #stopwords=STOPWORDS,
                      background_color='white',
                      ).generate_from_frequencies(dict(data))
plt.figure(figsize=(16,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# 평점 전처리 함수
def star_preprocessing(value) : 
#     value = int(text)

    if value <= 3 :        # 평점이 3점 이상 일때, 아닐때 
        return '0'
    else :
        return '1'      # 평점이 3점 이상일 경우 1이 return 되서 긍정이라는 판단 하도록 함

# 형태소 분석을 위한 함수 tokenizer를 통해서 글자 하나하나 자름
def tokenizer(text) :
    okt = Okt()         # 텍스트가 들어가면 okt에서 형태소를 분해함
    return okt.morphs(text)

In [ ]:
def step1_data_preprocessing() :
    # 수집한 데이터를 읽어온다.
    df = pd.read_excel('cos38.xlsx')

    # 전처리를 수행한다.
    df['rating'] = df['rating'].apply(star_preprocessing) # apply안에는 함수가 들어간다. 여기서는 star_preprocessing 함수가 들어감
        # 이 함수를 거치면 점수가 1.2.3점은 0점 (부정) , 4.5점은 1점 (긍정)으로 바뀐다
        
      # 학습데이터와 테스트 데이터로 나눈다.
    text_list = df['content'].tolist()
    star_list = df['rating'].tolist()

    from sklearn.model_selection import train_test_split

    # 80%는 학습, 20%는 test
    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.2, random_state=0)

    return text_train, text_test, star_train, star_test

In [ ]:
def step2_learning(X_train, y_train, X_test, y_test):
    # 주어진 데이터를 단어 사전으로 만들고 각 단어의 빈도수를 계산한 후 벡터화 하는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
        # 쪼갠단어를 긍정의 확률이 얼마 인지 부정이 얼마인지를 나타내줌 
        
    # 문장별 나오는 단어수 세서 수치화, 벡터화해서 학습을 시킨다.
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])
                    # logistic regression = 긍정, 부정 판단
    
    # pipeline = tfidf 와 logistic을 함께 쓰겠다는 것을 표현 
    
    # 학습한다.
    pipe.fit(X_train, y_train)

    # 학습 정확도 측정
    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))

    # 학습된 모델을 저장한다.
    with open('pipe.dat', 'wb') as fp :
        pickle.dump(pipe, fp)
                # pickle 형식으로 저장
    print('저장완료')

In [ ]:
def step3_using_model() :
    # 객체를 복원한다.
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)


    import numpy as np

    while True :
        text = input('리뷰를 작성해주세요 :')
        if text == '끝':
            print('프로그램 종료')
            break
        else: 
            str = [text]
            # 예측 정확도
            r1 = np.max(pipe.predict_proba(str) * 100)

            # 예측 결과
            r2 = pipe.predict(str)[0]
                                
            if r2 == '1' :
                print('긍정적인 리뷰')
            else :
                print('부정적인 리뷰')

            print('정확도 : %.3f' % r1)

        
# 학습 함수
def learning() :
    text_train, text_test, star_train, star_test = step1_data_preprocessing()
    step2_learning(text_train, star_train, text_test, star_test)

# 사용 함수
def using() :
    step3_using_model()

In [ ]:
learning()

In [ ]:
using()